In [1]:
!pip install redis
!pip install line-bot-sdk

In [2]:
"""

    1.針對跟redis的連線
    

"""

import redis

#製作redis連線
redis = redis.Redis(
    #redis container的host name
    host='redis',
    port=6379,
    #預設沒密碼
    password=None,
    #給格式
    charset="utf-8",
    #要解碼不然回傳的資料前面會多一個b
    decode_responses=True)


In [3]:
#redis.hmset("test",{"message":"88"})
redis.sadd("my_set", "87", "88", "89")

0

In [4]:
redis.scard("my_set")
redis.smembers("my_set")
"87" in redis.smembers("my_set")

True

In [5]:
!pip install python-logstash

  Running setup.py bdist_wheel for python-logstash ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/ae/be/2d/626d5c1fa358aada2de09a07c2318848ad35afada0f9651665
Successfully built python-logstash


In [6]:
import logging
import logstash
import sys

host = 'logstash'

test_logger = logging.getLogger('python-logstash-logger')
test_logger.setLevel(logging.INFO)

# UDP
# test_logger.addHandler(logstash.LogstashHandler(host, 12201, version=1))

# TCP
test_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

# test_logger.error('python-logstash: test logstash error message.')
# test_logger.info('python-logstash: test logstash info message.')
# test_logger.warning('python-logstash: test logstash warning message.')

# add extra field to logstash message
extra = {
    'test_string': 'python version: ' + repr(sys.version_info),
    'test_boolean': True,
    'test_dict': {'a': 1, 'b': 'c'},
    'test_float': 1.23,
    'test_integer': 123,
    'test_list': [1, 2, '3'],
}
test_logger.error('python-logstash: test extra fields', extra=extra)
test_logger.handlers.clear()
print('done,please see kibana')

done,please see kibana


In [7]:
import redis
from hashlib import md5


class SimpleHash(object):
    def __init__(self, cap, seed):
        self.cap = cap
        self.seed = seed

    def hash(self, value):
        ret = 0
        for i in range(len(value)):
            ret += self.seed * ret + ord(value[i])
        return (self.cap - 1) & ret


class BloomFilter(object):
    def __init__(self, host='redis', port=6379, db=0, blockNum=1, key='bloomfilter'):
        """
        :param host: the host of Redis
        :param port: the port of Redis
        :param db: witch db in Redis
        :param blockNum: one blockNum for about 90,000,000; if you have more strings for filtering, increase it.
        :param key: the key's name in Redis
        """
        self.server = redis.Redis(host=host, port=port, db=db)
        self.bit_size = 1 << 31  # Redis的String类型最大容量为512M，现使用256M
        self.seeds = [5, 7, 11, 13, 31, 37, 61]
        self.key = key
        self.blockNum = blockNum
        self.hashfunc = []
        for seed in self.seeds:
            self.hashfunc.append(SimpleHash(self.bit_size, seed))

    def isContains(self, str_input):
        if not str_input:
            return False
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        ret = True
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            ret = ret & self.server.getbit(name, loc)
        return ret

    def insert(self, str_input):
        m5 = md5()
        m5.update(str_input)
        str_input = m5.hexdigest()
        name = self.key + str(int(str_input[0:2], 16) % self.blockNum)
        for f in self.hashfunc:
            loc = f.hash(str_input)
            self.server.setbit(name, loc, 1)

In [12]:
import redis
from hashlib import md5

            
bf = BloomFilter()
url = "https://tw.news.appledaily.com/politics/realtime/20181125/1473059/sdasdasd"
url = url.encode('utf-8')
if bf.isContains(url):   # 判断字符串是否存在
    print('exists!')
else:
    print('not exists!')
    bf.insert(url)

exists!


In [13]:
import time

print(time.asctime())

Mon Dec 10 11:21:19 2018


In [12]:
import datetime
today = datetime.date.today()
today

datetime.date(2018, 12, 10)

In [2]:
import requests
import json

ip_location='chatbot_api'

News = {  
    "source": "蘋果日報",
    "label":"blue",
    "ariticle_emotion" : "positive",
    "title" : "「夜襲」激化綠支持者 陳其邁：明晚兩倍人數攻旗美",
    "url" : 'https://tw.news.appledaily.com//politics/realtime/20181110/1463926/',
    "date_" : '2018/11/10 12:15',
    "content" : '高雄市長選戰進入肉搏戰。繼國民黨韓國瑜陣營前天在美濃高美醫專舉辦宣稱逾3邁人造勢大會後，對手、民進黨籍陳其邁明晚也將在旗山體育場，舉辦造勢晚會拼場，陳其邁競選總部總協調陳啟昱表示，韓國瑜高美醫醫專場動員100輛遊覽車，「我們會加倍」，加上韓前晚唱「夜襲」軍歌，激化綠營支持者，明將有大批人潮湧進旗山體育場，陪陳其邁決戰大旗美。\xa0陳啟昱今與陳其邁競選總部總幹事、立委劉世芳合開記者會。陳啟昱說，韓國瑜高美醫專場，其支持者、高市農會理事長蕭漢俊稱動員100輛遊覽車，明天陳其邁在旗山體育場晚會，「我們會動員兩倍量」，且韓國瑜前晚唱「夜襲」，讓選情緊張，激化綠營支持者的心，「要參加（明晚）其邁造勢晚會詢問電話已大量湧入。」\xa0劉世芳說，韓國瑜的造勢晚會，成反年改陣營的歡樂派對，人多卻未必有選票。陳啟昱說，分析死忠「韓粉」，不少是反年改、反軍改成員，「這些人動員人力強」，但很多是從中北部而來。\xa0陳啟昱說，韓國瑜在晚會唱「夜襲」，明晚陳其邁晚會則請歌手許富凱、詹雅雯來演唱，氣氛決對和韓場不一樣。（吳慧芬／高雄報導）',
    "img_url" : 'https://tw.news.appledaily.com//politics/realtime/20181110',
    "kw": ['高雄選區', '陳其邁', '造勢', '旗山'],
    "author": ['吳慧芬'],
    "abstract": "高雄市長選戰進入肉搏戰。繼國民黨韓國瑜陣營前天在美濃高美醫專舉辦宣稱逾3邁人造勢大會後"
}

# 將json傳回API Server
Endpoint='http://%s:5000/news' % (ip_location)

# header要特別註明是json格式
Header={'Content-Type':'application/json'}

# 傳送post對API server新增資料 
Response=requests.post(Endpoint,headers=Header,data=json.dumps(News))

#印出Response的資料訊息
print(Response)
print(Response.text)

<Response [200]>
{
  "status_describe": "News https://tw.news.appledaily.com//politics/realtime/20181110/1463926/ is exist."
}



In [2]:
import requests
import json

ip_location='chatbot_api'

users = {
    "user_id" : "tyuetyok;tl,",
    "display_name" : "宗翰",
    "picture_url" : "https://tw.news.appledaily.com/",
    "status_message" : "張良榮好帥"
}

Endpoint='http://%s:5000/users' % (ip_location)

# header要特別註明是json格式
Header={'Content-Type':'application/json'}

# 傳送post對API server新增資料 
Response=requests.post(Endpoint,headers=Header,data=json.dumps(users))

#印出Response的資料訊息
print(Response)
print(Response.text)

<Response [200]>
{
  "status_describe": "success add user"
}



In [70]:
import requests
import json

ip_location='chatbot_api'

Endpoint='http://%s:5000/users_id' % (ip_location)

Response=requests.get(Endpoint)

print(Response.text)



[
  "tyuetyok;tl,", 
  "U7bd8f7a1c9e46ae01e0a06f6b2b8fb27"
]



In [56]:
import requests
import json

ip_location='chatbot_api'
playload = {"userid" : "U7bd8f7a1c9e46ae01e0a06f6b2b8fb27"}
Endpoint='http://%s:5000/users/%s'% (ip_location, playload["userid"])

Response=requests.get(Endpoint)
print(Response.url)

print(Response.text)


http://chatbot_api:5000/users/U7bd8f7a1c9e46ae01e0a06f6b2b8fb27
{
  "display_name": "\u826f\u69ae", 
  "join_datetime": "Mon, 10 Dec 2018 10:06:37 GMT", 
  "picture_url": "https://profile.line-scdn.net/0m0454be2c7251432be899969ec8d168bc5c6dd86192a1", 
  "status_message": null, 
  "user_id": "U7bd8f7a1c9e46ae01e0a06f6b2b8fb27"
}



取出所有User的所有資料

In [71]:
import requests
import json

ip_location='chatbot_api'

Endpoint='http://%s:5000/users' % (ip_location)

Response=requests.get(Endpoint)

print(Response.text)

[
  {
    "display_name": "\u5b97\u7ff0", 
    "join_datetime": "Mon, 10 Dec 2018 18:10:19 GMT", 
    "picture_url": "https://tw.news.appledaily.com/", 
    "status_message": "\u5f35\u826f\u69ae\u597d\u5e25", 
    "user_id": "tyuetyok;tl,"
  }, 
  {
    "display_name": "\u826f\u69ae", 
    "join_datetime": "Mon, 10 Dec 2018 10:06:37 GMT", 
    "picture_url": "https://profile.line-scdn.net/0m0454be2c7251432be899969ec8d168bc5c6dd86192a1", 
    "status_message": null, 
    "user_id": "U7bd8f7a1c9e46ae01e0a06f6b2b8fb27"
  }
]

